# Air Quality Time Series Analysis: Paris PM2.5 & SO2

**Portfolio Project 1: Air Quality Time Series Analysis**

This notebook demonstrates:
- Data acquisition from EEA API
- Data cleaning and validation
- Time series analysis
- Statistical analysis
- Visualization with matplotlib

**Data Source:** European Environment Agency (EEA)
**Location:** Paris, France
**Period:** January 2023
**Pollutants:** PM2.5, SO2

## 1. Setup and Data Download

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import requests
import zipfile
from scipy import stats

# Configure matplotlib
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

In [ ]:
def download_eea_data(country_code, city, pollutant, start_date, end_date):
    """
    Download air quality data from EEA API.
    
    Parameters:
    -----------
    country_code : str
        Two-letter country code (e.g., 'FR')
    city : str
        City name (e.g., 'Paris')
    pollutant : str
        Pollutant code (e.g., 'PM2.5', 'SO2')
    start_date : str
        Start date in ISO format
    end_date : str
        End date in ISO format
    
    Returns:
    --------
    Path
        Path to extracted data directory
    """
    api_url = "https://eeadmz1-downloads-api-appservice.azurewebsites.net/ParquetFile"
    
    request_body = {
        "countries": [country_code],
        "cities": [city],
        "pollutants": [pollutant],
        "dataset": 1,
        "dateTimeStart": start_date,
        "dateTimeEnd": end_date,
        "aggregationType": "hour"
    }
    
    print(f"Downloading {pollutant} data for {city}...")
    
    try:
        response = requests.post(api_url, json=request_body, timeout=300)
        response.raise_for_status()
        
        # Save and extract ZIP file
        data_dir = Path('data/raw')
        data_dir.mkdir(parents=True, exist_ok=True)
        
        zip_path = data_dir / f"{city.lower()}_{pollutant.lower()}_data.zip"
        with open(zip_path, 'wb') as f:
            f.write(response.content)
        
        extract_path = data_dir / f"{city.lower()}_{pollutant.lower()}"
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
        
        print(f"✓ Downloaded and extracted to {extract_path}")
        return extract_path
        
    except Exception as e:
        print(f"✗ Error downloading data: {e}")
        return None

In [ ]:
# Download SO2 data for Paris (January 2023)
paris_so2_path = download_eea_data(
    country_code='FR',
    city='Paris',
    pollutant='SO2',
    start_date='2023-01-01T00:00:00.000Z',
    end_date='2023-01-31T23:59:59.000Z'
)

## 2. Data Loading and Cleaning

In [ ]:
def load_and_clean_data(data_path):
    """
    Load and clean air quality data from parquet files.
    
    Parameters:
    -----------
    data_path : Path
        Path to directory containing parquet files
    
    Returns:
    --------
    pandas.DataFrame
        Cleaned dataframe
    """
    # Find all parquet files
    parquet_files = list(Path(data_path).glob('**/*.parquet'))
    print(f"Found {len(parquet_files)} parquet file(s)")
    
    # Load all files
    dfs = []
    for file in parquet_files:
        df = pd.read_parquet(file)
        dfs.append(df)
    
    df = pd.concat(dfs, ignore_index=True)
    print(f"Loaded {len(df):,} raw measurements")
    
    # Clean data
    original_count = len(df)
    
    # Convert to numeric and handle invalid values
    df['Value'] = pd.to_numeric(df['Value'], errors='coerce')
    df = df[df['Value'] > -999]  # Remove invalid measurements
    df = df.dropna(subset=['Value'])
    
    # Remove outliers (3 standard deviations)
    z_scores = np.abs((df['Value'] - df['Value'].mean()) / df['Value'].std())
    df = df[z_scores < 3]
    
    # Process datetime
    df['DateTime'] = pd.to_datetime(df['Start'])
    df['Date'] = df['DateTime'].dt.date
    df['Hour'] = df['DateTime'].dt.hour
    df['DayOfWeek'] = df['DateTime'].dt.dayofweek
    
    # Sort by datetime
    df = df.sort_values('DateTime').reset_index(drop=True)
    
    removed = original_count - len(df)
    print(f"Cleaned: {len(df):,} measurements ({removed} removed)")
    
    return df

In [ ]:
# Load and clean data
if paris_so2_path:
    df = load_and_clean_data(paris_so2_path)
    
    # Display first few rows
    print("\nData preview:")
    display(df[['DateTime', 'Value', 'Unit', 'Hour', 'DayOfWeek']].head(10))

## 3. Basic Statistics

In [ ]:
# Calculate statistics
stats_summary = {
    'Mean': df['Value'].mean(),
    'Median': df['Value'].median(),
    'Std Dev': df['Value'].std(),
    'Min': df['Value'].min(),
    'Max': df['Value'].max(),
    '25th Percentile': df['Value'].quantile(0.25),
    '75th Percentile': df['Value'].quantile(0.75),
    '95th Percentile': df['Value'].quantile(0.95)
}

print("\n=== SO2 Concentration Statistics (µg/m³) ===")
for key, value in stats_summary.items():
    print(f"{key:20s}: {value:.2f}")

# WHO guideline (24-hour mean: 40 µg/m³)
who_guideline = 40
exceedances = (df['Value'] > who_guideline).sum()
exceedance_pct = (exceedances / len(df)) * 100
print(f"\nWHO Guideline Exceedances: {exceedances} ({exceedance_pct:.1f}%)")

## 4. Visualizations

In [ ]:
# Create figure with 4 subplots
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Paris SO2 Air Quality Analysis - January 2023', fontsize=16, fontweight='bold')

# 1. Time series
axes[0, 0].plot(df['DateTime'], df['Value'], linewidth=0.8, alpha=0.7, color='steelblue')
axes[0, 0].axhline(y=df['Value'].mean(), color='red', linestyle='--', label=f'Mean: {df["Value"].mean():.2f}')
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('SO2 Concentration (µg/m³)')
axes[0, 0].set_title('Time Series')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Distribution
axes[0, 1].hist(df['Value'], bins=50, color='steelblue', alpha=0.7, edgecolor='black')
axes[0, 1].axvline(df['Value'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df["Value"].mean():.2f}')
axes[0, 1].axvline(df['Value'].median(), color='green', linestyle='--', linewidth=2, label=f'Median: {df["Value"].median():.2f}')
axes[0, 1].set_xlabel('SO2 Concentration (µg/m³)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Distribution of SO2 Concentrations')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. Hourly patterns
hourly_avg = df.groupby('Hour')['Value'].agg(['mean', 'std']).reset_index()
axes[1, 0].plot(hourly_avg['Hour'], hourly_avg['mean'], marker='o', linewidth=2, color='steelblue')
axes[1, 0].fill_between(hourly_avg['Hour'], 
                        hourly_avg['mean'] - hourly_avg['std'],
                        hourly_avg['mean'] + hourly_avg['std'],
                        alpha=0.3, color='steelblue')
axes[1, 0].set_xlabel('Hour of Day')
axes[1, 0].set_ylabel('SO2 Concentration (µg/m³)')
axes[1, 0].set_title('Daily Pattern (Mean ± Std Dev)')
axes[1, 0].set_xticks(range(0, 24, 3))
axes[1, 0].grid(True, alpha=0.3)

# 4. Weekly patterns
day_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
daily_avg = df.groupby('DayOfWeek')['Value'].agg(['mean', 'std']).reset_index()
axes[1, 1].bar(daily_avg['DayOfWeek'], daily_avg['mean'], color='steelblue', alpha=0.7, edgecolor='black')
axes[1, 1].errorbar(daily_avg['DayOfWeek'], daily_avg['mean'], yerr=daily_avg['std'], 
                    fmt='none', ecolor='black', capsize=5)
axes[1, 1].set_xlabel('Day of Week')
axes[1, 1].set_ylabel('SO2 Concentration (µg/m³)')
axes[1, 1].set_title('Weekly Pattern (Mean ± Std Dev)')
axes[1, 1].set_xticks(range(7))
axes[1, 1].set_xticklabels(day_names)
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 5. Key Findings

In [ ]:
# Find peak and minimum hours
peak_hour = hourly_avg.loc[hourly_avg['mean'].idxmax(), 'Hour']
min_hour = hourly_avg.loc[hourly_avg['mean'].idxmin(), 'Hour']
peak_value = hourly_avg['mean'].max()
min_value = hourly_avg['mean'].min()

print("\n=== KEY FINDINGS ===")
print(f"\n1. Data Coverage:")
print(f"   - Total measurements: {len(df):,}")
print(f"   - Date range: {df['Date'].min()} to {df['Date'].max()}")
print(f"   - Data completeness: {(len(df) / (31 * 24)):.1%}")

print(f"\n2. Daily Patterns:")
print(f"   - Peak hour: {int(peak_hour):02d}:00 ({peak_value:.2f} µg/m³)")
print(f"   - Minimum hour: {int(min_hour):02d}:00 ({min_value:.2f} µg/m³)")
print(f"   - Daily variation: {(peak_value - min_value):.2f} µg/m³")

weekday_avg = df[df['DayOfWeek'] < 5]['Value'].mean()
weekend_avg = df[df['DayOfWeek'] >= 5]['Value'].mean()
print(f"\n3. Weekly Patterns:")
print(f"   - Weekday average: {weekday_avg:.2f} µg/m³")
print(f"   - Weekend average: {weekend_avg:.2f} µg/m³")
print(f"   - Difference: {abs(weekday_avg - weekend_avg):.2f} µg/m³")

print(f"\n4. Air Quality Assessment:")
print(f"   - Mean concentration: {df['Value'].mean():.2f} µg/m³")
print(f"   - Below WHO guideline: {100 - exceedance_pct:.1f}% of time")
if df['Value'].mean() < who_guideline:
    print(f"   - Status: Generally good air quality")
else:
    print(f"   - Status: Attention needed")

## 6. Save Processed Data

In [ ]:
# Save cleaned data
output_dir = Path('data/processed')
output_dir.mkdir(parents=True, exist_ok=True)
output_file = output_dir / 'paris_so2_cleaned.parquet'

df.to_parquet(output_file, index=False)
print(f"\nCleaned data saved to: {output_file}")
print(f"File size: {output_file.stat().st_size / 1024:.1f} KB")

---

## Summary

This analysis demonstrates:
- ✓ Data acquisition from EEA API
- ✓ Data cleaning and validation
- ✓ Time series analysis (trends, patterns)
- ✓ Statistical analysis (mean, percentiles, exceedances)
- ✓ Professional visualizations

**Skills demonstrated:** Python, pandas, data cleaning, time series analysis, matplotlib, API integration, statistical analysis